In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.nn import init
import copy
import numpy as np

import queue

In [16]:
gpu = False

In [17]:
class RND(nn.Module):
    def __init__(self,width = 240, height =256, channel = 3):
        super(RND,self).__init__()
        
        self.width = width
        self.height = height
        self.channel = channel
        
        self.predictor = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=8,
                stride=4),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=4,
                stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1),
            nn.LeakyReLU(),
            Flatten(),
            nn.Linear(46592, 512), # change
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
            )
        self.target = copy.deepcopy(self.predictor)
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

            if isinstance(p, nn.Linear):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

        for param in self.target.parameters():
            param.requires_grad = False
            
    def forward(self, state):
        target_feature = self.target(state)
        predict_feature = self.predictor(state)
        return predict_feature, target_feature

In [18]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
class PPO(nn.Module):
    def __init__(self,width = 240, height =256, channel = 3, action_dim = 7):
        self.width = width
        self.height = height
        self.channel = channel
        self.action_dim = action_dim
        super(PPO,self).__init__()
        self.memory = []
        
        self.basic = nn.Sequential(\
                    nn.Conv2d(in_channels = 3,\
                             out_channels = 32,\
                             kernel_size = 8,\
                              stride = 4),
                    nn.ReLU(),
                                   nn.Conv2d(in_channels = 32,\
                                           out_channels = 64,\
                                           kernel_size = 4,\
                                           stride = 2),\
                    nn.ReLU(),\
                    nn.Conv2d(in_channels = 64,\
                             out_channels = 64,\
                             kernel_size = 3,\
                             stride = 1),
                                   nn.ReLU(),\
                                   Flatten(),
                    nn.Linear(46592,256), #have to change       
                    nn.ReLU(),\
                    nn.Linear(256,448),
                    nn.ReLU()
                    )
        self.actor = nn.Sequential(\
                                  nn.Linear(448,448),\
                                  nn.ReLU(),\
                                  nn.Linear(448,self.action_dim)\
                                  )
        
        self.extrinsic_critic = nn.Linear(448,1)
        self.intrinsic_critic = nn.Linear(448,1)
        
        init.orthogonal_(self.extrinsic_critic.weight, 0.01)
        self.extrinsic_critic.bias.data.zero_()

        init.orthogonal_(self.intrinsic_critic.weight, 0.01)
        self.intrinsic_critic.bias.data.zero_()

        for i in range(len(self.actor)):
            if type(self.actor[i]) == nn.Linear:
                init.orthogonal_(self.actor[i].weight, 0.01)
                self.actor[i].bias.data.zero_()
    def forward(self, x,dim = -1):
        x = self.basic(x)
        action = self.actor(x)
        action_prob = F.softmax(action,dim = dim)
        
        intrinsic = self.intrinsic_critic(x)
        extrinsic = self.extrinsic_critic(x)
        return action_prob,extrinsic,intrinsic    

    


In [19]:
learning_rate = 0.0001
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 4
T_horizon     = 128
critic_coef = 0.5
ent_coef = 0.001
intrinsic_gamma = 0.99
extrinsic_gamma = 0.999
update_proportion = 0.25

extrinsic_advantage_coef = 2
intrinsic_advantage_coef = 1
class Agent(nn.Module):
    def __init__(self,width=240,height=256,channel = 3,action_dim=7,learning_rate=0.0005):
        self.width = width
        self.height = height
        self.channel = channel
        self.action_dim = action_dim
        
        super(Agent,self).__init__()
        
        self.memory = []
        self.intrinsic_queue = queue.Queue()
        self.intrinsic_input_queue = queue.Queue()
        self.ppo = PPO(self.width, self.height, self.channel, self.action_dim)
        self.rnd = RND(self.width, self.height , self.channel)
        self.optimizer = optim.Adam(self.parameters(),lr = learning_rate)
    def put_data(self,data):
        self.memory.append(data)
    def make_batch(self):
        state_list, action_list, extrinsic_reward_list, intrinsic_reward_list, next_state_list, \
        prob_list, extrinsic_done_list,intrinsic_done_list = [],[],[],[],[],[], [],[]
        for data in self.memory:
            state,action,extrinsic_reward, intrinsic_reward,next_state,prob,done = data
            state_list.append(state)
            action_list.append([action])
            extrinsic_reward_list.append([extrinsic_reward])
            intrinsic_reward_list.append([intrinsic_reward])

            next_state_list.append(next_state)
            extrinsic_done_mask = 0 if done else 1
            extrinsic_done_list.append([extrinsic_done_mask])
            intrinsic_done_list.append([1])
            prob_list.append([prob])
        self.memory = []

        s,a,er,ir,next_s,extrinsic_done_list,intrinsic_done_list,prob \
                                        = torch.tensor(state_list,dtype=torch.float),\
                                        torch.tensor(action_list),\
                                        torch.tensor(extrinsic_reward_list,dtype = torch.float),\
                                        torch.tensor(intrinsic_reward_list,dtype = torch.float),\
                                        torch.tensor(next_state_list,dtype=torch.float),\
                                        torch.tensor(extrinsic_done_list,dtype = torch.float),\
                                        torch.tensor(intrinsic_done_list,dtype = torch.float),\
                                        torch.tensor(prob_list,dtype = torch.float)
        if gpu:
            return s.cuda(),a.cuda(),er.cuda(),ir.cuda(),next_s.cuda(),extrinsic_done_list.cuda()\
            ,intrinsic_done_list.cuda(),prob.cuda() 
        else :
            return s,a,er,ir,next_s,extrinsic_done_list,intrinsic_done_list,prob  
    
    def train(self):
        state,action,extrinsic_reward,intrinsic_reward, next_state,extrinsic_done_list,\
        intrinsic_done_list,action_prob = self.make_batch()
        
        for i in range(K_epoch):
            state = state.squeeze()
            next_state = next_state.squeeze()
            predicted_action, predicted_extrinsic, predicted_intrinsic = self.ppo(state)
            predicted_next_action, predicted_next_extrinsic, predicted_next_intrinsic = self.ppo(next_state)

            if gpu:
                intrinsic_next_state_mean = torch.mean(torch.cat(list(self.intrinsic_input_queue.queue)),dim = 0).cuda()
                intrinsic_next_state_std = torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0).cuda()
            
                preprocessed_next_state = torch.clamp(((next_state - intrinsic_next_state_mean) / \
                                                   intrinsic_next_state_std + torch.tensor(1e-8).cuda()),-5,5)
            else:
                intrinsic_next_state_mean = torch.mean(torch.cat(list(self.intrinsic_input_queue.queue)),dim = 0)
                intrinsic_next_state_std = torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
            
                preprocessed_next_state = torch.clamp(((next_state - intrinsic_next_state_mean) / \
                                                   intrinsic_next_state_std + torch.tensor(1e-8)),-5,5)
            
            predict_feature, target_feature = self.rnd(next_state)
            td_error = extrinsic_reward + extrinsic_gamma * predicted_next_extrinsic * extrinsic_done_list
            delta = td_error - predicted_next_extrinsic
            if gpu:
                delta = delta.detach().cpu().numpy()
            else:
                delta = delta.detach().numpy()
            advantage_list = []
            
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_list.append([advantage])
            advantage_list.reverse()
            if gpu:
                advantage = torch.tensor(advantage_list,dtype = torch.float).cuda()
            else:
                advantage = torch.tensor(advantage_list,dtype = torch.float)
            ##intrinsic_advantage
            intrinsic_td_error = intrinsic_reward + intrinsic_gamma * predicted_next_intrinsic * intrinsic_done_list
            intrinsic_delta = intrinsic_td_error - predicted_next_intrinsic
            if gpu:
                intrinsic_delta = intrinsic_delta.detach().cpu().numpy()
            else:
                intrinsic_delta = intrinsic_delta.detach().numpy()
                
            intrinsic_advantage_list = []
            intrinsic_advantage = 0.0
            
            for intrinsic_delta_t in intrinsic_delta[::-1]:
                intrinsic_advantage = gamma * lmbda * intrinsic_advantage + intrinsic_delta_t[0]
                intrinsic_advantage_list.append([intrinsic_advantage])
            intrinsic_advantage_list.reverse()
            if gpu:
                intrinsic_advantage = torch.tensor(intrinsic_advantage_list,dtype = torch.float).cuda()
            else:
                intrinsic_advantage = torch.tensor(intrinsic_advantage_list,dtype = torch.float)
            #### intrinsic_error
            intrinsic_error = (intrinsic_td_error - predicted_intrinsic.detach()).pow(2)
            if gpu:
                masking = torch.rand(len(intrinsic_error)).cuda()
                masking = (masking < update_proportion).type(torch.FloatTensor).cuda()
            else:
                masking = torch.rand(len(intrinsic_error))
                masking = (masking < update_proportion).type(torch.FloatTensor)
            if gpu:
                intrinsic_error = (intrinsic_error * masking).sum() / torch.max(intrinsic_error.sum(), torch.Tensor([1]).cuda())
            else:
                intrinsic_error = (intrinsic_error * masking).sum() / torch.max(masking.sum(), torch.Tensor([1]))

            
            now_action = predicted_action
            m = Categorical(now_action)
            entropy = m.entropy().mean()
            
            
            now_action = now_action.gather(1,action)
            
            
            ratio = torch.exp(torch.log(now_action) - torch.log(action_prob))
            advantage = extrinsic_advantage_coef * advantage +  intrinsic_advantage_coef * intrinsic_advantage
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio , 1-eps_clip, 1 + eps_clip) * advantage
            loss = - torch.min(surr1,surr2) + critic_coef * (F.smooth_l1_loss(predicted_extrinsic,td_error.detach()) +\
                    intrinsic_error) - ent_coef * entropy + F.mse_loss(predict_feature, target_feature)
            
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

In [20]:
epochs = 1000
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


In [21]:
if gpu:
    model = Agent().cuda()
else:
    model = Agent()


In [8]:
'''
try:
    if gpu:
        model.load_state_dict(torch.load("weights/2_episode_model"))
    else:
        model.load_state_dict(torch.load("weights/2_episode_model", map_location={'cuda:0': 'cpu'}))
except :
    pass
'''

In [40]:
T_horizon = 20

for epoch in range(epochs):
    global_step = 0
    model.intrinsic_queue = queue.Queue()
    model.intrinsic_input_queue = queue.Queue()
    state = env.reset()
    state = np.array(state)/255
            #state = np.transpose(state,(2,0,1))
    state = np.moveaxis(state, -1, 0)
    state = torch.tensor(state).float()
    state = state.unsqueeze(0)
    done = False
    while not done :
        for t in range(T_horizon):
            #env.render()
            global_step +=1

            if gpu:
                action_prob, _ , _ = model.ppo.forward(state.cuda())
            else:
                action_prob, _ , _ = model.ppo.forward(state)
            m = Categorical(action_prob)
            action = m.sample().item()
            
            
            next_state, extrinsic_reward, done, info = env.step(action)
            next_state = np.array(next_state)/255
            next_state = np.moveaxis(next_state,-1,0)
            next_state = torch.tensor(next_state).float()
            next_state = next_state.unsqueeze(0)
            
            
            model.intrinsic_input_queue.put(next_state)
            if len(model.intrinsic_input_queue.queue) > 128:
                model.intrinsic_input_queue.get()
            intrinsic_next_state_mean = \
                    torch.mean(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
            if len(model.intrinsic_input_queue.queue) == 1:
                intrinsic_next_state_std = torch.zeros(1)
            else:
                intrinsic_next_state_std = \
                        torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
    
            preprocessed_next_state = \
                    torch.clamp(((next_state - intrinsic_next_state_mean) / (intrinsic_next_state_std + 1e-8)), -5,5)
            
            
            #(model.intrinsic_input_queue)
            if gpu:
                predictor,target = model.rnd.forward(preprocessed_next_state.cuda())
            else:
                predictor,target = model.rnd.forward(preprocessed_next_state)
            intrinsic_reward = (predictor - target).pow(2).sum(1) / 2
            if len(model.intrinsic_queue.queue) > 128:
                model.intrinsic_queue.get()
            model.intrinsic_queue.put(intrinsic_reward.item())
            intrinsic_mean = np.mean(model.intrinsic_queue.queue)
            intrinsic_std = np.std(model.intrinsic_queue.queue)
            intrinsic_reward = (intrinsic_reward - intrinsic_mean) / (intrinsic_std+ 1e-8)
            
            if info['time'] == 0 or info['life'] == 1 or info['time'] == 300:
                done = True
                reward = -10.

            model.put_data((state.tolist(), action, extrinsic_reward/100, (intrinsic_reward.item()), next_state.tolist(), action_prob[0][action].item(), done))
            print('global_step : ',global_step,', action : ', action,'action_prob : ',action_prob.tolist()[0])
            print('extrinsic_reward : ',extrinsic_reward,'intrinsic_reward : ',intrinsic_reward.item())
            print('place',info['x_pos'])
            print('info',info)
            if done :
                print('epoch : ',epoch, ', global_step : ',global_step)
                break
            state = next_state
        model.train()
    #env.render()

#env.close()

intrinsic_reward.item() :  0.0
global_step :  1 , action :  3 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  0 intrinsic_reward :  0.0
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
intrinsic_reward.item() :  0.0
global_step :  2 , action :  2 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  0 intrinsic_reward :  0.0
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
intrinsic_reward.item() :  1.9368853569030762
global_step :  3 , action :  1 action_prob :  [0.14285677671432495, 0.

intrinsic_reward.item() :  22939.35546875
global_step :  19 , action :  0 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285694062709808]
extrinsic_reward :  -1 intrinsic_reward :  2.2297821044921875
place 48
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 48, 'x_pos_screen': 48, 'y_pos': 79}
intrinsic_reward.item() :  31892.9140625
global_step :  20 , action :  3 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  1 intrinsic_reward :  2.7246546745300293
place 49
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 49, 'x_pos_screen': 49, 'y_pos': 79}
intrinsic_reward.item() :  37568.2734375
global_step :  21 , a

intrinsic_reward.item() :  42682.17578125
global_step :  37 , action :  0 action_prob :  [0.12560036778450012, 0.12549930810928345, 0.1909065693616867, 0.12511563301086426, 0.12487104535102844, 0.184056356549263, 0.1239507645368576]
extrinsic_reward :  1 intrinsic_reward :  1.0110849142074585
place 65
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 65, 'x_pos_screen': 65, 'y_pos': 91}
intrinsic_reward.item() :  15310.85546875
global_step :  38 , action :  6 action_prob :  [0.12560328841209412, 0.12550222873687744, 0.19089840352535248, 0.12511862814426422, 0.1248740553855896, 0.1840495616197586, 0.12395390123128891]
extrinsic_reward :  1 intrinsic_reward :  -0.5207064747810364
place 66
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 66, 'x_pos_screen': 66, 'y_pos': 86}
intrinsic_reward.item() :  15174.3798828125
global_step :  39 , act

KeyboardInterrupt: 

In [22]:
model.intrinsic_queue.queue

deque([nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan])

In [20]:
preprocessed_next_state

tensor([[[[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [-5.,  5.,  5.,  ...,  5.,  5., nan],
          [-5.,  5.,  5.,  ...,  5., nan, nan],
          [ 5., nan, nan,  ..., nan, nan,  5.]],

         [[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [-5., -5., -5.,  ..., -5., -5., nan],
          [-5., -5., -5.,  ..., -5., nan, nan],
          [-5., nan, nan,  ..., nan, nan, -5.]],

         [[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]]])

319

In [44]:
env.close()

ValueError: env has already been closed.

In [22]:
if gpu:
    model.load_state_dict(torch.load("weights/3"))
else:
    model.load_state_dict(torch.load("weights/3", map_location={'cuda:0': 'cpu'}))

In [23]:
epochs = 1000
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT



In [34]:
env = gym_super_mario_bros.make('SuperMarioBros-1-3-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [35]:
T_horizon = 20

for epoch in range(epochs):
    global_step = 0
    state = env.reset()
    state = np.array(state)/255
            #state = np.transpose(state,(2,0,1))
    state = np.moveaxis(state, -1, 0)
    state = torch.tensor(state).float()
    state = state.unsqueeze(0)
    done = False
    while not done :
        for t in range(T_horizon):
            env.render()
            global_step +=1

            if gpu:
                action_prob, _ , _ = model.ppo.forward(state.cuda())
            else:
                action_prob, _ , _ = model.ppo.forward(state)
            m = Categorical(action_prob)
            action = m.sample().item()
            next_state, extrinsic_reward, done, info = env.step(action)
            next_state = np.array(next_state)/255
            next_state = np.moveaxis(next_state,-1,0)
            next_state = torch.tensor(next_state).float()
            next_state = next_state.unsqueeze(0)
            if gpu:
                predictor,target = model.rnd.forward(next_state.cuda())
            else:
                predictor,target = model.rnd.forward(next_state)
            intrinsic_reward = (predictor - target).pow(2).sum(1) / 2
            if info['time'] == 0 :
                done = True
                reward = -10.
            if done :
                print('epoch : ',epoch, ', global_step : ',global_step)
                break
            print('global_step : ',global_step,', action : ', action,'action_prob : ',action_prob.tolist()[0])
            print('extrinsic_reward : ',extrinsic_reward,'intrinsic_reward : ',intrinsic_reward.item())
            print('place',info['x_pos'])
            print('info',info)
            state = next_state
    #env.render()

#env.close()

global_step :  1 , action :  4 action_prob :  [0.05983036383986473, 0.17653553187847137, 0.0441148541867733, 0.5566235184669495, 0.0725478008389473, 0.042902544140815735, 0.04744540527462959]
extrinsic_reward :  0 intrinsic_reward :  411.56402587890625
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 300, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
global_step :  2 , action :  2 action_prob :  [0.05983036383986473, 0.17653553187847137, 0.0441148541867733, 0.5566235184669495, 0.0725478008389473, 0.042902544140815735, 0.04744540527462959]
extrinsic_reward :  0 intrinsic_reward :  411.56402587890625
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 300, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 84}
global_step :  3 , action :  1 action_prob :  [0.05983036383986473, 0.17653553187847137, 0.0441148541867733, 0.5566235184669495, 0.0725478008389473, 0.0

global_step :  21 , action :  3 action_prob :  [0.059839654713869095, 0.17654338479042053, 0.0441221185028553, 0.556575357913971, 0.07255519181489944, 0.04291249439120293, 0.04745174199342728]
extrinsic_reward :  1 intrinsic_reward :  411.8468322753906
place 51
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 299, 'world': 1, 'x_pos': 51, 'x_pos_screen': 51, 'y_pos': 79}
global_step :  22 , action :  3 action_prob :  [0.05985831096768379, 0.1765560656785965, 0.04413874074816704, 0.5564744472503662, 0.07257462292909622, 0.042928896844387054, 0.0474688820540905]
extrinsic_reward :  2 intrinsic_reward :  409.69305419921875
place 53
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 299, 'world': 1, 'x_pos': 53, 'x_pos_screen': 53, 'y_pos': 79}
global_step :  23 , action :  3 action_prob :  [0.05986681208014488, 0.17656221985816956, 0.04414632171392441, 0.5564278364181519, 0.07258360087871552,

global_step :  43 , action :  0 action_prob :  [0.05975869297981262, 0.1764753758907318, 0.04406150057911873, 0.556989848613739, 0.07249054312705994, 0.042827147990465164, 0.04739689826965332]
extrinsic_reward :  3 intrinsic_reward :  419.9659423828125
place 95
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 298, 'world': 1, 'x_pos': 95, 'x_pos_screen': 86, 'y_pos': 97}
global_step :  44 , action :  3 action_prob :  [0.06068625673651695, 0.17709366977214813, 0.04486773908138275, 0.5520557165145874, 0.07341953366994858, 0.04365895688533783, 0.04821815341711044]
extrinsic_reward :  2 intrinsic_reward :  399.4931335449219
place 97
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 298, 'world': 1, 'x_pos': 97, 'x_pos_screen': 87, 'y_pos': 92}
global_step :  45 , action :  3 action_prob :  [0.05847126618027687, 0.17557556927204132, 0.042946331202983856, 0.5638759136199951, 0.07119212299585342

global_step :  64 , action :  3 action_prob :  [0.05714671313762665, 0.174601748585701, 0.041814640164375305, 0.5709572434425354, 0.0698714479804039, 0.040495578199625015, 0.045112691819667816]
extrinsic_reward :  3 intrinsic_reward :  428.06732177734375
place 144
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 297, 'world': 1, 'x_pos': 144, 'x_pos_screen': 106, 'y_pos': 110}
global_step :  65 , action :  3 action_prob :  [0.05670202150940895, 0.17425698041915894, 0.04143271595239639, 0.5733663439750671, 0.06941680610179901, 0.04010433703660965, 0.04472067952156067]
extrinsic_reward :  3 intrinsic_reward :  421.8553466796875
place 147
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 297, 'world': 1, 'x_pos': 147, 'x_pos_screen': 107, 'y_pos': 108}
global_step :  66 , action :  3 action_prob :  [0.057204511016607285, 0.17464035749435425, 0.04186017066240311, 0.5706691145896912, 0.0699200

global_step :  86 , action :  3 action_prob :  [0.05457090958952904, 0.17261597514152527, 0.039589665830135345, 0.584944486618042, 0.06718721985816956, 0.038283947855234146, 0.042807694524526596]
extrinsic_reward :  3 intrinsic_reward :  450.08953857421875
place 210
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 296, 'world': 1, 'x_pos': 210, 'x_pos_screen': 112, 'y_pos': 101}
global_step :  87 , action :  3 action_prob :  [0.05509524047374725, 0.17303602397441864, 0.040041107684373856, 0.582086443901062, 0.06773441284894943, 0.03873194381594658, 0.043274879455566406]
extrinsic_reward :  3 intrinsic_reward :  446.8307189941406
place 213
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 296, 'world': 1, 'x_pos': 213, 'x_pos_screen': 112, 'y_pos': 97}
global_step :  88 , action :  3 action_prob :  [0.05430394411087036, 0.17238500714302063, 0.03936558589339256, 0.5863998532295227, 0.066918

...

extrinsic_reward :  3 intrinsic_reward :  447.98126220703125
place 278
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 293, 'world': 1, 'x_pos': 278, 'x_pos_screen': 112, 'y_pos': 63}
global_step :  148 , action :  3 action_prob :  [0.05510670691728592, 0.1730078160762787, 0.04004213958978653, 0.5821004509925842, 0.06772929430007935, 0.038741689175367355, 0.04327185079455376]
extrinsic_reward :  3 intrinsic_reward :  431.1484375
place 281
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 293, 'world': 1, 'x_pos': 281, 'x_pos_screen': 112, 'y_pos': 59}
global_step :  149 , action :  3 action_prob :  [0.05534571781754494, 0.1731952279806137, 0.04024358093738556, 0.5808191299438477, 0.06796853244304657, 0.03895222768187523, 0.04347562789916992]
extrinsic_reward :  3 intrinsic_reward :  429.13250732421875
place 284
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'stat

global_step :  10 , action :  6 action_prob :  [0.05961605906486511, 0.17639197409152985, 0.043925948441028595, 0.557774543762207, 0.07232643663883209, 0.04271500185132027, 0.04724998399615288]
extrinsic_reward :  1 intrinsic_reward :  413.97406005859375
place 41
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 300, 'world': 1, 'x_pos': 41, 'x_pos_screen': 41, 'y_pos': 100}
global_step :  11 , action :  4 action_prob :  [0.05967261642217636, 0.17643171548843384, 0.0439753383398056, 0.5574703812599182, 0.07238389551639557, 0.0427655465900898, 0.04730050638318062]
extrinsic_reward :  0 intrinsic_reward :  412.70538330078125
place 41
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 300, 'world': 1, 'x_pos': 41, 'x_pos_screen': 41, 'y_pos': 103}
global_step :  12 , action :  1 action_prob :  [0.059678398072719574, 0.17643551528453827, 0.0439801886677742, 0.5574402809143066, 0.072389416396617

global_step :  30 , action :  3 action_prob :  [0.05982242524623871, 0.17653192579746246, 0.04410692676901817, 0.5566681623458862, 0.07253710180521011, 0.042897697538137436, 0.04743576422333717]
extrinsic_reward :  2 intrinsic_reward :  413.2981872558594
place 68
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 299, 'world': 1, 'x_pos': 68, 'x_pos_screen': 68, 'y_pos': 96}
global_step :  31 , action :  3 action_prob :  [0.05980929359793663, 0.17652267217636108, 0.04409528523683548, 0.5567394495010376, 0.07252340763807297, 0.042886052280664444, 0.04742375388741493]
extrinsic_reward :  3 intrinsic_reward :  414.3782958984375
place 71
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 299, 'world': 1, 'x_pos': 71, 'x_pos_screen': 71, 'y_pos': 100}
global_step :  32 , action :  3 action_prob :  [0.05989186838269234, 0.1765792965888977, 0.04416729509830475, 0.5562973022460938, 0.072607189416885

global_step :  51 , action :  1 action_prob :  [0.057804934680461884, 0.1750953048467636, 0.04238089174032211, 0.5674107074737549, 0.07053947448730469, 0.04107797518372536, 0.04569076746702194]
extrinsic_reward :  3 intrinsic_reward :  434.695556640625
place 128
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 298, 'world': 1, 'x_pos': 128, 'x_pos_screen': 97, 'y_pos': 106}
global_step :  52 , action :  1 action_prob :  [0.05807214230298996, 0.17528285086154938, 0.04261305555701256, 0.5659791827201843, 0.07081256061792374, 0.04131057485938072, 0.045929569751024246]
extrinsic_reward :  3 intrinsic_reward :  428.8128662109375
place 131
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 3, 'status': 'small', 'time': 298, 'world': 1, 'x_pos': 131, 'x_pos_screen': 98, 'y_pos': 109}
global_step :  53 , action :  3 action_prob :  [0.058218106627464294, 0.1753910630941391, 0.04273540526628494, 0.5652058720588684, 0.07095336914

KeyboardInterrupt: 

In [36]:
env.close()